<p align= " center"> <strong>SPATIAL ERROR REGRESSION MODEL</p>

<p><b>By: Jefferson C.</b></p>

---

### __SER Model (SEM)__

__Def.__

Modelo econométrico espacial usado para __modelar la dependencia espacial en el término de error__ entre diferentes ubicaciones geográficas.  
Se asume que el error en una ubicación está influenciado por los errores en ubicaciones cercanas (según la estructura de vecindad definida por $W$).

Model

- __Forma General__

$$
y_i = x_i' \beta + \varepsilon_i
$$

$$
\varepsilon_i = \lambda \sum_{j=1}^{n} w_{ij}\,\varepsilon_j + u_i,
\qquad
u_i \sim \text{iid}(0,\sigma^2)
$$

donde:

- $y_i$: variable dependiente en la unidad espacial $i$.
- $x_i$: vector $k \times 1$ de regresores (incluye constante si aplica).
- $\beta$: vector $k \times 1$ de parámetros.
- $w_{ij}$: elemento de la matriz de pesos espaciales $W$.
- $\lambda$: parámetro de autocorrelación espacial en el error.
- $u_i$: choque “puro” no espacial (ruido blanco).

Consideremos:

- __Forma Matricial__

$$
y = X\beta + \varepsilon
$$

$$
\varepsilon = \lambda W \varepsilon + u,
\qquad
u \sim (0, \sigma^2 I_n)
$$

donde:

- $y$ es $n \times 1$,
- $X$ es $n \times k$,
- $\varepsilon$ y $u$ son $n \times 1$,
- $W$ es $n \times n$.

In [56]:
# IMPORT DATAA
import geopandas as gpd

data = gpd.read_file("../Data Bases/airbnb")
data.isna().sum()

community      0
shape_area     0
shape_len      1
AREAID         0
response_r     9
accept_r      10
rev_rating    10
price_pp       8
room_type      8
num_spots      0
poverty        0
crowded        0
dependency     0
without_hs     0
unemployed     0
income_pc      0
harship_in     0
num_crimes     0
num_theft      0
population     0
geometry       0
dtype: int64

In [57]:
# SER MODEL
from pysal.lib import weights
from spreg import ML_Error

# Weight Matrix
W = weights.Queen.from_dataframe(data, use_index=True)
W.transform = 'r'

# Dependent Variable 
Y = data['income_pc']

# Independent Variable
X =data[['crowded','num_crimes']]

# Model 
model = ML_Error(Y,X,w=W)
print(model.summary)

# Residuals
data['RESIDUALS'] = model.u

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ML SPATIAL ERROR (METHOD = full)
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   income_pc                Number of Observations:          77
Mean dependent var  :  25563.1688                Number of Variables   :           3
S.D. dependent var  :  15293.0983                Degrees of Freedom    :          74
Pseudo R-squared    :      0.2892
Log likelihood      :   -808.0860
Sigma-square ML     :60774727.8809                Akaike info criterion :    1622.172
S.E of regression   :   7795.8148                Schwarz criterion     :    1629.204

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT     34922.21961   

c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(


### __Assumptions__

### 1. __Stationarity__

__Este supuesto se refiere a estabilidad (invertibilidad) del proceso espacial del error.__ Esto garantiza que el error espacial tenga media y varianza bien definidas y que no “explote” por la retroalimentación espacial.

__Mathematical Formulation__

La condición clave es que el operador sea invertible:

$$
(I_n-\lambda W)\ \text{es no singular}
$$

Equivalente a la restricción sobre $\lambda$:

$$
|\lambda| < \frac{1}{\rho(W)}
$$

donde $\rho(W)$ es el radio espectral de $W$ (máximo valor absoluto de sus eigenvalues).

__Hyphotesis__

- $H_0)$ No hay estabilidad / no hay invertibilidad.

  $$
  H_0:\ |\lambda|\ \ge\ \frac{1}{\rho(W)}
  $$

- $H_1)$ Hay estabilidad / invertibilidad.

  $$
  H_1:\ |\lambda| < \frac{1}{\rho(W)}
  $$

__Test__

En la práctica, no se usa Dickey-Fuller. La “prueba” de este supuesto se maneja como:

- **Chequeo de admisibilidad:** al estimar SEM, se restringe $\lambda$ al rango que hace invertible $(I_n-\lambda W)$.
- (Complemento habitual) Se suele contrastar **dependencia espacial en el error** con:
  $$  H_0:\ \lambda = 0  \quad\text{vs}\quad  H_1:\ \lambda \neq 0  $$
  mediante **LM-error / Robust LM-error**.

In [ ]:
# LM-ERROR / ROBUST LM-ERROR (SEM/SER diagnostics)

import spreg

# OLS (spreg adds constant automatically)
ols = spreg.OLS(Y, X, w=W, name_y="y", name_x=[f"x{i+1}" for i in range(X.shape[1])])

# LM-error + Robust LM-error
lms = spreg.diagnostics_sp.LMtests(ols, W, tests=["lme", "rlme"])

# LM-error
lm_err_stat, lm_err_p = lms.lme       
# Robust LM-error
rlm_err_stat, rlm_err_p = lms.rlme    

print(f"LM-error (LM)        : {lm_err_stat:.4f}")
print(f"p-value              : {lm_err_p:.4f}\n")

print(f"Robust LM-error (RLM): {rlm_err_stat:.4f}")
print(f"p-value              : {rlm_err_p:.4f}\n")

# Decision
alpha = 0.05
if rlm_err_p < alpha:
    print("Reject H0: evidence of spatial dependence in the error (SEM/SER recommended).")
else:
    print("Fail to reject H0: no evidence of spatial error dependence.")

LM-error (LM)        : 57.1089
p-value              : 0.0000

Robust LM-error (RLM): 4.6179
p-value              : 0.0316

Reject H0: evidence of spatial dependence in the error (SEM/SER recommended).


### 2. __Independence of errors__

__La independencia no se exige para $\varepsilon$ (porque $\varepsilon$ incorpora dependencia espacial), sino para el componente “puro” $u$.__ Es decir, una vez controlada la estructura espacial del error, las innovaciones $u_i$ no deben presentar correlación entre unidades espaciales.

__Mathematical Formulation__

En forma matricial:

$$
\mathbb{E}(u\mid X)=0,
\qquad
\text{Var}(u\mid X)=\sigma^2 I_n
$$

En forma escalar (para $i\neq j$):

$$
\text{Cov}(u_i,u_j\mid X)=0
$$

__Hypothesis__

- $H_0)$ **No hay independencia** (existe autocorrelación espacial remanente en $u$).

  $$
  H_0:\ \exists\, i\neq j\ \text{tal que}\ \text{Cov}(u_i,u_j\mid X)\neq 0
  $$

- $H_1)$ **Hay independencia** (no hay autocorrelación espacial en $u$).

  $$
  H_1:\ \text{Cov}(u_i,u_j\mid X)=0\ \ \forall\, i\neq j
  $$

__Test__

En práctica, se verifica que **no quede autocorrelación espacial en los residuos “filtrados”**:

- Calcular $\hat u$ (innovaciones estimadas) a partir de los residuales del modelo SEM, por ejemplo:
  $$
  \hat u = (I_n-\hat\lambda W)\,\hat\varepsilon
  $$
- Aplicar **Moran’s I** (u otra prueba de autocorrelación espacial) sobre $\hat u$.
  - Si el test **no es significativo**, es consistente con independencia de $u$.
  - Si es significativo, sugiere dependencia espacial no capturada (violación del supuesto).

In [82]:
# MORAN'S I ON FILTERED INNOVATIONS (u_hat) AFTER SEM/SER

import numpy as np
import spreg
import libpysal
from esda.moran import Moran

# Fit SEM/SER (ML)
sem = spreg.ML_Error(Y, X, w=W, name_y="y", name_x=[f"x{i+1}" for i in range(X.shape[1])])
lam_hat = float(sem.lam)

# Residuals
ehat = getattr(sem, "u", None)
ehat = np.asarray(data["RESIDUALS"]).reshape(-1, 1)

# Filtered innovations (I-lambda_hat W)
Wmat = W.sparse if hasattr(W, "sparse") else W
uhat = ehat - lam_hat * (Wmat @ ehat)

# Moran's I on u_hat
w_moran = W if hasattr(W, "neighbors") else libpysal.weights.WSP(Wmat).to_W()
w_moran.transform = "r"

mi = Moran(uhat.flatten(), w=w_moran, permutations=999)

# Output
print("MORAN'S I (Filtered innovations u) \n")

print(f"I      : {mi.I:.6f}")
print(f"p-value: {mi.p_sim:.6g}")
print(f"z_sim  : {mi.z_sim:.6f}\n")

alpha = 0.05
if mi.p_sim < alpha:
    print("Reject H0: remaining spatial autocorrelation in û (possible misspecification).\n\n")
else:
    print("Fail to reject H0: û is consistent with spatial independence.\n\n")

MORAN'S I (Filtered innovations u) 

I      : 0.079149
p-value: 0.095
z_sim  : 1.314111

Fail to reject H0: û is consistent with spatial independence.




c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(


### 3. __Homoscedasticiity__

__La homocedasticidad se asume para el componente no espacial $u$ (no para $\varepsilon$).__ Es decir, la varianza de las innovaciones $u_i$ debe ser constante entre unidades espaciales; la heterogeneidad espacial sistemática no debe quedar en $u$.

__Mathematical Formulation__

En forma matricial:

$$
\text{Var}(u\mid X)=\sigma^2 I_n
$$

En forma escalar (para todo $i$):

$$
\text{Var}(u_i\mid X)=\sigma^2
$$

y, de forma equivalente:

$$
\text{Var}(u_i\mid X)=\text{Var}(u_j\mid X)\quad \forall\, i\neq j
$$

__Hypothesis__

- $H_0)$ **Heterocedasticidad** en $u$ (varianza no constante).

  $$
  H_0:\ \exists\, i \ \text{tal que}\ \text{Var}(u_i\mid X)\neq \sigma^2
  $$

- $H_1)$ **Homocedasticidad** en $u$ (varianza constante).

  $$
  H_1:\ \text{Var}(u_i\mid X)=\sigma^2\ \ \forall\, i
  $$

__Test__

En práctica, se evalúa heterocedasticidad sobre las **innovaciones/residuos filtrados** $\hat u$:

- Obtener $\hat u$ (por ejemplo):
  $$
  \hat u = (I_n-\hat\lambda W)\,\hat\varepsilon
  $$
- Aplicar pruebas estándar de heterocedasticidad (p.ej., **Breusch–Pagan** o **White**) usando $\hat u$ como residual.
  - Si el test **no es significativo**, es consistente con homocedasticidad en $u$.
  - Si es significativo, sugiere heterocedasticidad (violación del supuesto).

In [87]:
# BREUSCH–PAGAN TEST (on u_hat)

import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan

# Design matrix (aux regression)
X_bp = sm.add_constant(np.asarray(X), has_constant="add")

# Residuals / innovations (u_hat)
u = np.asarray(uhat).flatten()

# Test
bp_lm, bp_p, bp_f, bp_fp = het_breuschpagan(u, X_bp)

# Output
print("BREUSCH-PAGAN TEST \n")
print(f"LM stat  : {bp_lm:.4f}")
print(f"p-value  : {bp_p:.4f}")
print(f"F stat   : {bp_f:.4f}")
print(f"p-value  : {bp_fp:.4f}\n")

# Decision
alpha = 0.05
if bp_p < alpha:
    print("Reject H0: evidence of heteroskedasticity.")
else:
    print("Fail to reject H0: no evidence of heteroskedasticity.")

BREUSCH-PAGAN TEST 

LM stat  : 5.8561
p-value  : 0.0535
F stat   : 3.0456
p-value  : 0.0536

Fail to reject H0: no evidence of heteroskedasticity.


In [90]:
# WHITE TEST (on u_hat)

import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white

# Design matrix (aux regression)
X_w = sm.add_constant(np.asarray(X), has_constant="add")

# Residuals / innovations (u_hat)
u = np.asarray(uhat).flatten()

# Test
w_lm, w_p, w_f, w_fp = het_white(u, X_w)

# Output
print("WHITE TEST \n")

print(f"LM stat  : {w_lm:.4f}")
print(f"p-value  : {w_p:.4f}")
print(f"F stat   : {w_f:.4f}")
print(f"p-value  : {w_fp:.4f}\n")

# Decision
alpha = 0.05
if w_p < alpha:
    print("Reject H0: evidence of heteroscedasticity.")
else:
    print("Fail to reject H0: no evidence of heteroscedasticity.")

WHITE TEST 

LM stat  : 12.9376
p-value  : 0.0240
F stat   : 2.8677
p-value  : 0.0205

Reject H0: evidence of heteroscedasticity.


### 4. __No multicollinearity__

__Este supuesto se refiere a las covariables $X$ (no al componente espacial del error).__ Se requiere que $X$ no tenga combinaciones lineales exactas entre sus columnas, para que $\beta$ sea identificable y la estimación sea posible.

__Mathematical Formulation__

Condición de rango completo:

$$\text{rank}(X)=k$$

Equivalente:

$$X'X\ \text{es invertible} \qquad\Longleftrightarrow\qquad \det(X'X)\neq 0$$

(“No existe” un vector $a\neq 0$ tal que $Xa=0$).

__Hypothesis__

- $H_0)$ Existe multicolinealidad perfecta (no rango completo).

  $$  H_0:\ \text{rank}(X)<k  $$

- $H_1)$ No existe multicolinealidad perfecta (rango completo).

  $$  H_1:\ \text{rank}(X)=k  $$

__Test__

En práctica se evalúa **multicolinealidad alta** (casi colinealidad) con diagnósticos sobre $X$:

- **VIF** (Variance Inflation Factor) por regresor.
- **Número de condición** (condition number) de $X$ o de $X'X$.
- **Matriz de correlaciones** y/o eigenvalores pequeños de $X'X$.

Si VIF es muy alto o el número de condición es grande, hay evidencia de multicolinealidad severa (aunque no necesariamente perfecta).

In [93]:
# MULTICOLLINEARITY (Rank / Condition Number / VIF)

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Regressors
X_const = sm.add_constant(X, has_constant="add")

# Rank(X) (perfect multicollinearity)
rank_X = np.linalg.matrix_rank(X_const.values)
k = X_const.shape[1]

print("MULTICOLLINEARITY \n")
print(f"Rank(X) : {rank_X}")
print(f"k       : {k}\n")

if rank_X < k:
    print("Perfect multicollinearity detected: rank(X) < k.\n")
else:
    print("No perfect multicollinearity: full rank.\n")

# Condition number (high - strong multicollinearity)
cond_num = np.linalg.cond(X_const.values)
print(f"Condition number : {cond_num:.4f}")

if cond_num > 30:
    print("Warning: high multicollinearity (condition number > 30).\n")
else:
    print("OK: no severe multicollinearity by condition number.\n")

# VIF table
vif_data = pd.DataFrame()
vif_data["Variable"] = X_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]

print("VIF TABLE \n")
print(vif_data)

MULTICOLLINEARITY 

Rank(X) : 3
k       : 3

No perfect multicollinearity: full rank.

Condition number : 17788.8669

VIF TABLE 

     Variable       VIF
0       const  3.862254
1     crowded  1.013940
2  num_crimes  1.013940


### 5. __Exogeneity__

__En SEM (SER), la exogeneidad exige que las covariables $X$ sean independientes del componente no espacial $u$.__ Es decir, los regresores no deben estar correlacionados con las innovaciones “puras” del error; la dependencia espacial se modela en $\varepsilon$, pero el origen del problema de endogeneidad es la correlación entre $X$ y $u$.

__Mathematical Formulation__

Exogeneidad (estricta) condicional:

$$\mathbb{E}(u\mid X)=0$$

Forma equivalente en momentos:

$$\mathbb{E}(X'u)=0$$

En escalar, para cada regresor $x_{ri}$:

$$\mathbb{E}(x_{ri}\,u_i)=0\qquad (r=1,\dots,k;\ i=1,\dots,n)$$

__Hypothesis__

- $H_0)$ Endogeneidad (algún regresor está correlacionado con $u$).

  $$  H_0:\ \exists\, r \ \text{tal que}\ \mathbb{E}(x_{r}'u)\neq 0  $$

- $H_1)$ Exogeneidad (ningún regresor está correlacionado con $u$).

  $$  H_1:\ \mathbb{E}(X'u)=0  $$

__Test__

No existe una “prueba universal” solo con la muestra. En práctica se usan:

- **Pruebas de endogeneidad con IV** (Durbin–Wu–Hausman) si hay instrumentos válidos.
- Diagnóstico de **variables omitidas**, **simultaneidad** o **error de medición** (fuentes típicas de endogeneidad).
- Si se sospecha endogeneidad espacial (por ejemplo, $Wy$ omitido o variables espaciales omitidas), se comparan especificaciones (SEM vs SAR/SDM) y se consideran métodos con instrumentos/2SLS/GS2SLS.

Por limitaciones de este curso no veremos este supuesto a profundidad

---